In [6]:
import numpy as np
from sentence_transformers import SentenceTransformer
import pandas as pd

# Embedding Vector 추출에 활용할 모델(jhgan/ko-sbert-sts) 불러오기
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

c:\Users\minkyu\AppData\Local\anaconda3\envs\sd2\lib\site-packages\sentence_transformers\SentenceTransformer.py:195: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(


In [1]:
import pandas as pd
df = pd.read_csv('train_df.csv', encoding = 'utf-8-sig')


In [2]:
df['공사종류(대분류)'] = df['공사종류'].str.split(' / ').str[0]
df['공사종류(중분류)'] = df['공사종류'].str.split(' / ').str[1]
df['공종(대분류)'] = df['공종'].str.split(' > ').str[0]
df['공종(중분류)'] = df['공종'].str.split(' > ').str[1]
df['사고객체(대분류)'] = df['사고객체'].str.split(' > ').str[0]
df['사고객체(중분류)'] = df['사고객체'].str.split(' > ').str[1]

In [3]:
df.value_counts('공종(중분류)')

공종(중분류)
철근콘크리트공사        6353
가설공사            2268
기타              1859
해체 및 철거공사        933
기계설비공사           845
토공사              816
철골공사             703
건축물 부대공사         581
수장공사             534
관공사              497
건축 토공사           496
미장공사             482
전기설비공사           456
타일 및 돌공사         404
도로 및 포장공사        362
교량공사             310
조적공사             297
도장공사             266
터널공사             259
산업설비공사           256
목공사              247
금속공사             239
창호 및 유리공사        218
하천공사             203
방수공사             190
관공사 부대공사         177
항만공사             148
지붕 및 홈통공사        144
말뚝공사             106
철도 및 궤도공사        104
지정공사              86
조경공사              58
지반개량공사            45
강구조물공사            41
지반조사              33
프리캐스트 콘크리트공사      19
댐 및 제방공사          17
특수 건축물공사           9
통신설비공사             8
Name: count, dtype: int64

In [23]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

grouped = df.groupby("사고객체(대분류)")
# 결과 저장을 위한 딕셔너리
avg_similarities = {}

# 각 그룹에 대해 평균 코사인 유사도 계산
for category, group in grouped:
    sentences = group['재발방지대책 및 향후조치계획'].tolist()
    
    # 그룹 내 문장이 2개 미만이면 계산할 수 없으므로 0 또는 np.nan 처리
    if len(sentences) < 2:
        avg_similarities[category] = np.nan
        continue
    
    embeddings = model.encode(sentences, convert_to_tensor=False)
    
    sim_matrix = cosine_similarity(embeddings)
    
    # 대각선은 자기 자신과의 유사도로 제외 (k=1부터 시작)
    n = sim_matrix.shape[0]
    upper_tri_indices = np.triu_indices(n, k=1)
    pairwise_sims = sim_matrix[upper_tri_indices]
    
    avg_similarity = np.mean(pairwise_sims)
    avg_similarities[category] = avg_similarity

# 결과 DataFrame 생성
result_df = pd.DataFrame(list(avg_similarities.items()), columns=['공종(중분류)', '평균유사도'])
print(result_df)

   공종(중분류)     평균유사도
0      가시설  0.550008
1     건설공구  0.540466
2     건설기계  0.536110
3     건설자재  0.541783
4       기타  0.514122
5       부재  0.526603
6      시설물  0.536595
7       질병  0.486281
8  토사 및 암반  0.525183


In [25]:
overall_avg = np.nanmean(list(avg_similarities.values()))
print("전체 평균 유사도:", overall_avg)

전체 평균 유사도: 0.5285723


In [9]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

grouped = df.groupby("공사종류(대분류)")
# 결과 저장을 위한 딕셔너리
avg_similarities = {}

# 각 그룹에 대해 평균 코사인 유사도 계산
for category, group in grouped:
    sentences = group['재발방지대책 및 향후조치계획'].tolist()
    
    # 그룹 내 문장이 2개 미만이면 계산할 수 없으므로 0 또는 np.nan 처리
    if len(sentences) < 2:
        avg_similarities[category] = np.nan
        continue
    
    embeddings = model.encode(sentences, convert_to_tensor=False)
    
    sim_matrix = cosine_similarity(embeddings)
    
    # 대각선은 자기 자신과의 유사도로 제외 (k=1부터 시작)
    n = sim_matrix.shape[0]
    upper_tri_indices = np.triu_indices(n, k=1)
    pairwise_sims = sim_matrix[upper_tri_indices]
    
    avg_similarity = np.mean(pairwise_sims)
    avg_similarities[category] = avg_similarity

# 결과 DataFrame 생성
result_df = pd.DataFrame(list(avg_similarities.items()), columns=['공사종류(대분류)', '평균유사도'])
print(result_df)

KeyboardInterrupt: 

In [17]:
overall_avg = np.nanmean(list(avg_similarities.values()))
print("전체 평균 유사도:", overall_avg)

전체 평균 유사도: 0.53173584
